### 라이브러리 준비


**라이브러리 설치**   
- !pip install selenium  
- !pip install openpyxl  
- !pip install pandas  
- !pip install beautifulsoup4   


In [8]:
# 위치 확인
import os
os.getcwd()

'c:\\Users\\woo\\Desktop\\crowling'

# 에버겟 상품 정보 가져오기

필요한 데이터 : 대표이미지, 상품명, 판매가, 옵션수, 상세페이지 링크

!! 옵션 수는 페이지에서 상품 클릭 후, 확인 할 수 있어서 코드 찾기 귀찮음 + 잘 모르겠움 에 이슈로 하지 않음...데헷(。・ω・。)

In [16]:
import selenium
from selenium import webdriver as wd
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests

from itertools import repeat
import re

In [20]:
url = 'http://everget.co.kr/shop/listtype.php?type=0'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

# soup에 해당 url 페이지의 parsing된 html 정보 모두 담기게 됨

In [54]:
soup

<!DOCTYPE html>
<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="no" http-equiv="imagetoolbar"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>전체상품 | (주)에버겟  - 카카오프렌즈 문구/팬시/리빙  제품 총판업체</title>
<link href="http://everget.co.kr/theme/basic/css/default_shop.css?ver=221112" rel="stylesheet"/>
<link href="http://everget.co.kr/js/font-awesome/css/font-awesome.min.css?ver=221112" rel="stylesheet"/>
<link href="http://everget.co.kr/js/owlcarousel/owl.carousel.css?ver=221112" rel="stylesheet"/>
<link href="http://everget.co.kr/theme/basic/skin/outlogin/shop_basic/style.css?ver=221112" rel="stylesheet"/>
<link href="http://everget.co.kr/skin/latest/mainimages/style.css?ver=221112" rel="stylesheet"/>
<link href="http://everget.co.kr/skin/latest/mainimages/jquery.bxslider.css?ver=221112" rel="stylesheet"/>
<link href="http://everget.co.kr/skin/latest/basic/style.css?ver=221112" rel="stylesheet"/>
<link href="http://everget.co.kr/skin/shop/basic/style.css?ver=221112" r

### 01
상세페이지 링크, 상품명, 대표이미지 값 찾기

In [48]:
html = soup.find_all('div', attrs={'class':'sct_img'})
html

[<div class="sct_img">
 <a href="http://everget.co.kr/shop/item.php?it_id=9990262">
 <img alt="LF 러블리 피규어 연필깎이" height="225" src="http://everget.co.kr/data/item/9990262/thumb-1_225x225.jpg" width="225"/>
 </a>
 <div class="sct_btn list-10-btn">
 <button class="btn_cart sct_cart" data-it_id="9990262" type="button"></button>
 </div>
 <div class="cart-layer"></div>
 </div>,
 <div class="sct_img">
 <a href="http://everget.co.kr/shop/item.php?it_id=9990261">
 <img alt="춘식이 비치 타월 철푸덕 XL" height="225" src="http://everget.co.kr/data/item/9990261/thumb-1_225x225.png" width="225"/>
 </a>
 <div class="sct_btn list-10-btn">
 <button class="btn_cart sct_cart" data-it_id="9990261" type="button"></button>
 </div>
 <div class="cart-layer"></div>
 </div>,
 <div class="sct_img">
 <a href="http://everget.co.kr/shop/item.php?it_id=9990260">
 <img alt="스마일 문구세트" height="225" src="http://everget.co.kr/data/item/9990260/thumb-7Iqk66eI7J28_225x225.png" width="225"/>
 </a>
 <div class="sct_btn list-10-btn">
 <

In [50]:
# 상세페이지 링크, 상품명, 대표이미지
for div_tag in soup.find_all('div', class_='sct_img'):
    a_tag = div_tag.find('a')
    href = a_tag['href']
    img_tag = a_tag.find('img')
    alt = img_tag['alt']
    src = img_tag['src']
    
    # 확인용
    print("href:", href)
    print("alt:", alt)
    print("src:", src)

href: http://everget.co.kr/shop/item.php?it_id=9990262
alt: LF 러블리 피규어 연필깎이
src: http://everget.co.kr/data/item/9990262/thumb-1_225x225.jpg
href: http://everget.co.kr/shop/item.php?it_id=9990261
alt: 춘식이 비치 타월 철푸덕 XL
src: http://everget.co.kr/data/item/9990261/thumb-1_225x225.png
href: http://everget.co.kr/shop/item.php?it_id=9990260
alt: 스마일 문구세트
src: http://everget.co.kr/data/item/9990260/thumb-7Iqk66eI7J28_225x225.png
href: http://everget.co.kr/shop/item.php?it_id=9990259
alt: 해피 문구세트
src: http://everget.co.kr/data/item/9990259/thumb-KakaoTalk_20240423_104730161_225x225.png
href: http://everget.co.kr/shop/item.php?it_id=9990258
alt: 러블리 문구세트
src: http://everget.co.kr/data/item/9990258/thumb-KakaoTalk_20240423_104708618_225x225.png
href: http://everget.co.kr/shop/item.php?it_id=9990257
alt: 춘식이 문구세트 A
src: http://everget.co.kr/data/item/9990257/thumb-KakaoTalk_20240423_104819195_225x225.png
href: http://everget.co.kr/shop/item.php?it_id=9990256
alt: 춘식이 문구세트 B
src: http://everget.co.

### 02

판매가 찾기

In [58]:
for div_tag in soup.find_all('div', class_='sct_cost'):
    cost = div_tag.get_text(strip=True)
    print("Cost:", cost)

Cost: 4,500원
Cost: 19,800원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 1원
Cost: 6,500원
Cost: 2,000원
Cost: 2,500원


### 03
전 페이지 상품 모두 가져오기

> http://everget.co.kr/shop/listtype.php?&type=0&sort=&**page=1**
> http://everget.co.kr/shop/listtype.php?&type=0&sort=&**page=2**

In [67]:
start = 1
title_list = []
img_list = []
cost_list = []
url_list = []

while start < 24:  # 1페이지~23페이지
    try:
        url = 'http://everget.co.kr/shop/listtype.php?&type=0&sort=&page={}'.format(start)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')

        for div_tag_1 in soup.find_all('div', class_='sct_img'):
            a_tag = div_tag_1.find('a')
            href = a_tag['href']  # 상세페이지 링크
            img_tag = a_tag.find('img')
            alt = img_tag['alt']  # 상품명
            src = img_tag['src']  # 대표이미지

            title_list.append(alt)  # 상품명
            img_list.append(src)  # 대표이미지
            url_list.append(href)  # 상세페이지 링크

        for div_tag_2 in soup.find_all('div', class_='sct_cost'):
            cost = div_tag_2.get_text(strip=True)  # 판매가

            cost_list.append(cost)  # 판매가

        start += 1

    except Exception as e:
        print("Error occurred:", e)
        break

In [69]:
df = pd.DataFrame({'상품명': title_list,'판매가' : cost_list,
                  '대표이미지': img_list, 'url': url_list})
df.to_csv('에버겟.csv', encoding = 'utf-8-sig')

### 데이터 확인하기

In [71]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('에버겟.csv')

# 처음 5개 행 출력
print(df.head())

   Unnamed: 0               상품명      판매가  \
0           0   LF 러블리 피규어 연필깎이   4,500원   
1           1  춘식이 비치 타월 철푸덕 XL  19,800원   
2           2          스마일 문구세트       1원   
3           3           해피 문구세트       1원   
4           4          러블리 문구세트       1원   

                                               대표이미지  \
0  http://everget.co.kr/data/item/9990262/thumb-1...   
1  http://everget.co.kr/data/item/9990261/thumb-1...   
2  http://everget.co.kr/data/item/9990260/thumb-7...   
3  http://everget.co.kr/data/item/9990259/thumb-K...   
4  http://everget.co.kr/data/item/9990258/thumb-K...   

                                                url  
0  http://everget.co.kr/shop/item.php?it_id=9990262  
1  http://everget.co.kr/shop/item.php?it_id=9990261  
2  http://everget.co.kr/shop/item.php?it_id=9990260  
3  http://everget.co.kr/shop/item.php?it_id=9990259  
4  http://everget.co.kr/shop/item.php?it_id=9990258  
